In [76]:
get_prices_per_state = """
    SELECT AVG(p.price) as price, state
    FROM `prices` as p
    JOIN `hospitals` as h
    ON p.npi_number = h.npi_number
    WHERE p.code = "27130"
    GROUP BY h.state
"""

In [12]:
from doltpy_integrations.metaflow.dolt import DoltConfig, DoltDT

from doltpy.cli import Dolt
from doltpy.cli.read import read_pandas_sql

dolt = Dolt("hospital-price-transparency")

In [77]:
df = read_pandas_sql(dolt, get_prices_per_state)

In [85]:
df.price = df.price.astype(int)

In [45]:
import chartify

ch = chartify.Chart()
ch.set_title("Horizontal bar plot")
ch.set_subtitle("Horizontal with color grouping")
ch.plot.bar(
    data_frame=df,
    categorical_columns='state',
    numeric_column='price',
        color_column=None)
# ch.show('png')

ValueError: failed to validate FigureOptions(...).active_drag: expected an element of either Auto, String or Instance(Drag), got None

In [52]:
import altair as alt
alt.Chart(df).mark_bar().encode(
    x='state',
    y='price'
)

alt.Chart(...)

In [86]:
import pandas as pd
low_source = df.sort_values(by="price").iloc[:5,:]
high_source = df.sort_values(by="price", ascending=False).iloc[:5,:]
fl = df[df.state == "FL"]
source = pd.concat([high_source, low_source, fl]).sort_values(by="price")
bars = alt.Chart(source).mark_bar().encode(
    y='state:N',
    x="price:Q"
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='price:Q'
)

(bars + text).properties(height=300)

alt.LayerChart(...)

In [ ]:
bars = alt.Chart(source[source.price]).mark_bar().encode(
    y='state:N',
    x="price:Q"
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='price:Q'
)

(bars + text).properties(height=300)

In [92]:
from doltpy_integrations.metaflow.dolt import DoltConfig, DoltDT

from doltpy.cli import Dolt
from doltpy.cli.read import read_pandas

d_age = Dolt("state-age")
age_df = read_pandas(d_age, "age")
age_df.age = age_df.age.astype(float)
age_df.state = age_df.state.astype(str)

In [93]:
age_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_full  51 non-null     object 
 1   state       51 non-null     object 
 2   age         51 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.3+ KB


In [95]:
source.state = source.state.astype(str)

In [100]:
join_df = age_df.set_index("state").join(df.set_index("state")).dropna()

In [117]:
alt.Chart(join_df).mark_point().encode(
    x=alt.X('age', scale=alt.Scale(domain=(30, 50))),
    y=alt.Y('price',scale=alt.Scale(domain=(0, 400000)))
).transform_regression('age', 'price',method="linear"
).mark_line(color="red")

alt.Chart(...)

In [119]:
chart = alt.Chart(join_df).mark_point().encode(
    x=alt.X('age', scale=alt.Scale(domain=(30, 50))),
    y=alt.Y('price',scale=alt.Scale(domain=(0, 400000)))
)
    
(
    chart +
    chart.transform_regression('age','price',method="linear").mark_line(color="red")
)

alt.LayerChart(...)